# Trabalho Prático I - Problema de Transporte de Objeto

Descrição na [ementa](enunciado_trabalho_I_v2.pdf).

Neste cenário, um agente deve percorrer a grade 7x6, encontrar o objeto e transportá-lo até na base. Essa tarefa deve ser executada na menor quantidade de passos detempo possível. O agente não possui nenhum conhecimento prévio sobre o ambiente, o qual possui paredes, as quais ele não pode transpor. O agente também não possui conhecimento prévio sobre a localização do objeto. A localização inicial do agente, disposição das paredes e objeto são sempre fixas, conforme indicado na ilustração. A cada passo de tempo, o agente pode executar os seguintes movimentos na grade:
- mover para cima, baixo, esquerda ou direita;
- permanecer na mesma célula;

Este cenário apresenta algumas restrições de movimentação:
- O agente pode realizar apenas uma movimentação por passo de tempo;
- Se o agente escolher se mover para uma célula que não está vazia, seja por conta de uma parede ou objeto, ele não se move, i.e., permanece na mesma célula;
- Qualquer tentativa de locomoção para além da grade, resultará na não movimentação do agente;
- O objeto sé pode ser agarrado pela sua esquerda ou direita;
- Quando o agente ́e posicionado à direita ou esquerda do objeto, o objeto ée agarrado automaticamente;
- Uma vez agarrado o objeto, o agente não pode soltá-lo;
- O agente, quando agarrado ao objeto, só consegue se mover para uma nova célula desde que não haja nenhuma restrição de movimentação para o agente e objeto;

O episódio ée concluído automaticamente quando o objeto entra na base ou se atingir um número máximo de passos de tempo sem resolver a tarefa. Em ambos os casos, um novo episódio ́é iniciado, com o agente e objeto situados conforme abaixo.

In [242]:
# Libs necessária
import gym
from gym.envs.toy_text import discrete

import numpy as np

#import random
import sys

In [256]:
# Montando o cenário 
# Exemplo 1: https://towardsdatascience.com/creating-a-custom-openai-gym-environment-for-stock-trading-be532be3910e
# Exemplo 2: https://github.com/caburu/gym-cliffwalking/blob/master/gym_cliffwalking/envs/cliffwalking_env.py
# Exemplo 3: https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py

class ObjectTransportEnv(discrete.DiscreteEnv):
    """
    O mapa é descrito com:
    _ : caminho livre
    o : posição inicial do agente
    + : objeto a ser transportado
    $ : objetivo
    # : bloqueio
    O episódio termina chegar no objetivo.
    Sua recompensa é 1 se pegar o objeto e 1 se chegar ao objetivo com o objeto.
    Fora do mapa não é acessível. Mais regras na ementa acima.
    """

    metadata = {"render.modes": ["human", "ansi"]}
    l_free_path = b'_'
    l_agent = b'o'
    l_object = b'+'
    l_goal = b'$'
    l_wall = b'#'
    actions_movements = [
        (0, -1) # left
        ,(-1, 0) # up
        ,(0, 1) # rigth
        ,(1, 0) # down
    ]
    steps = []

    def __init__(self, desc):
        self.desc = np.asarray(desc, dtype="c")
        self.nrow, self.ncol = self.desc.shape
        self.reward_range = (-100, 100)

        # 4 ações (cima, baixo, esquerda, direita)
        nA = len(self.actions_movements)
        # Espaço é o tamanho do mapa
        nS = self.nrow * self.ncol

        # estado inicial
        isd = np.array(self.desc == self.l_agent).astype("float64").ravel()
        isd /= isd.sum()

        # lista com transições (probability, nextstate, reward, done)
        P = {s: {} for s in range(nS)}
        
        for row in range(self.nrow):
            for col in range(self.ncol):
                s = self._to_s(row, col)
                # movimento protegido para não ir pra fora da lista
                #for action, predicted_pos in self._possible_actions(row, col):
                for action, predicted_pos in self._possible_actions(row, col):
                    P[s][action] = [(1.0, *self._update_probability_matrix(predicted_pos))]

        super(ObjectTransportEnv, self).__init__(nS, nA, P, isd)

    def _inc(self, row, col, action):
        inc_row, inc_col = action
        col = col + inc_col
        row = row + inc_row
        return (row, col)

    def _to_s(self, row, col):
        return row * self.ncol + col

    def _s_to_row_col(self, s = None):
        _s = s or self.s
        return _s // self.ncol, _s % self.ncol

    def _possible_actions(self, row, col):
        for index in range(len(self.actions_movements)):
            newpos = self._inc(row, col, self.actions_movements[index])
            if self._is_valid_pos(newpos):
                yield index, newpos
    
    def _is_valid_pos(self, pos):
        row, col = pos
        inside = row >= 0 and col >= 0 and row < self.nrow and col < self.ncol
        return inside

        # Teste pra ver se é melhor ignorar a parede ou dar uma pontuação negativa
        #walls = False
        #if inside:
        #    walls = self.desc[row, col] == self.l_wall
        #return inside and not walls

    def is_valid_current_action(self, action):
        return action in self.P[self.s]

    def sample_from_available_current_actions(self):
        keys = list(self.P[self.s].keys())
        value = self.np_random.choice(keys)
        return value

    def _update_probability_matrix(self, predicted_pos):
        newrow, newcol = predicted_pos
        newstate = self._to_s(newrow, newcol)
        newletter = self.desc[newrow, newcol]
        done = bytes(newletter) in [self.l_goal]
        #done = bytes(newletter) in [self.l_goal, self.l_wall]
        #reward = float(newletter == self.l_goal 
        #    and newcol < (self.ncol-1)
        #    and self.desc[newrow, newcol+1] == self.l_object)        
        rewards = {
            self.l_goal: 100.0,
            self.l_wall: -100.0,
        }
        reward = rewards.get(newletter, -1)
        return newstate, reward, done


    def reset(self):
        super().reset()
        self.steps = [self.s] # para renderizar o caminho

    def step(self, a):
        result = super().step(a)
        self.steps.append(self.s)
        return result

    def render(self, mode="human"):
        outfile = StringIO() if mode == "ansi" else sys.stdout

        desc = self.desc.tolist()
        desc2 = desc[:]
        colors = {
            self.l_goal: 'blue',
            self.l_agent: 'red',
            self.l_wall: 'white',
            self.l_object: 'magenta',
        }
        for row in range(len(desc)):
            for col in range(len(desc[row])):
                sb = desc[row][col]
                color = colors.get(sb)
                ss = sb.decode('utf-8')
                desc2[row][col] = gym.utils.colorize(ss, color, highlight=True) if color else ss

        for _s in self.steps:
            row, col = self._s_to_row_col(_s)
            color = colors.get(desc[row][col])
            walked = "~"
            desc2[row][col] = gym.utils.colorize(walked, color, highlight=True) if color else walked
        desc2[row][col] = gym.utils.colorize(self.l_agent.decode('utf-8'), colors.get(self.l_agent), highlight=True)

        if self.lastaction is not None:
            outfile.write(
                "  ({})\n".format(["Left", "Down", "Right", "Up"][self.lastaction])
            )
        else:
            outfile.write("\n")
        outfile.write("\n".join(" ".join(line) for line in desc2) + "\n")

        if mode != "human":
            with closing(outfile):
                return outfile.getvalue()

t1_map = [
    "__$$$__",
    "___#___",
    "___+___",
    "_______",
    "##_####",
    "o_____#"
]

t1_env = ObjectTransportEnv(t1_map)
t1_env.reset()
t1_env.render()



_ _ $ $ $ _ _
_ _ _ # _ _ _
_ _ _ + _ _ _
_ _ _ _ _ _ _
# # _ # # # #
o _ _ _ _ _ #


In [251]:
def Qlearning(environment, num_episodes=100, alpha=0.3, gamma=0.9, epsilon=1.0, decay_epsilon=0.1, max_epsilon=1.0, min_epsilon=0.01):
  
  # initializing the Q-table
  Q = np.zeros((environment.observation_space.n, environment.action_space.n))
  
  # additional lists to keep track of reward and epsilon values
  rewards = []
  epsilons = []
  last_accumulated_reward = -999999

  # episodes
  for episode in range(num_episodes):
      
      # reset the environment to start a new episode
      state = environment.reset()

      # reward accumulated along episode
      accumulated_reward = 0
      
      # steps within current episode
      for step in range(100):
          action = None

          # epsilon-greedy action selection
          # exploit with probability 1-epsilon
          if np.random.uniform(0, 1) > epsilon:
              action = np.argmax(Q[state,:])
              if not environment.is_valid_current_action(action):
                action = None
              #else:
                #print('action1', ['left','up','right','down'][action], action, Q[state,:])
          # explore with probability epsilon
          if not action:
              #action = environment.action_space.sample()
              action = environment.sample_from_available_current_actions()
              #print('action2', ['left','up','right','down'][action], action, environment.P[environment.s])

          # perform the action and observe the new state and corresponding reward
          new_state, reward, done, info = environment.step(action)
          #print('state', new_state, 'reward', reward, 'done', done)

          # update the Q-table
          Q[state, action] = Q[state, action] + alpha * (reward + gamma * np.max(Q[new_state, :]) - Q[state, action])
          
          # update the accumulated reward
          accumulated_reward += reward
          #print(Q)

          # update the current state
          state = new_state

          # end the episode when it is done
          if done == True:
              break
      
      # decay exploration rate to ensure that the agent exploits more as it becomes experienced
      epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_epsilon*episode)
      
      # update the lists of rewards and epsilons
      rewards.append(accumulated_reward)
      epsilons.append(epsilon)

      # Teste pra mostrar cada caminho melhor
      #if accumulated_reward > last_accumulated_reward:
      #  environment.render()
      #  print('>' * 10, episode, last_accumulated_reward, '->', accumulated_reward)
      #  last_accumulated_reward = accumulated_reward 
      #break


  # render the environment
  environment.render()
  print(accumulated_reward)
    
  # return the list of accumulated reward along episodes
  return rewards

In [258]:
#num_episodes=100
num_episodes=10000
alpha=0.3
gamma=0.9
epsilon=1.0
decay_epsilon=0.1

# run Q-learning
rewards = Qlearning(t1_env, num_episodes, alpha, gamma, epsilon, decay_epsilon)

# print results
print ("Average reward (all episodes): " + str(sum(rewards)/num_episodes))
print ("Average reward (last 10 episodes): " + str(sum(rewards[-10:])/10))

  (Down)
_ _ $ $ o _ _
_ _ _ # ~ _ _
_ _ _ ~ ~ _ _
_ _ ~ ~ _ _ _
# # ~ # # # #
~ ~ ~ _ _ _ #
92.0
Average reward (all episodes): 13.2495
Average reward (last 10 episodes): 72.2
